In [1]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input, SpatialDropout1D, Dropout, CuDNNLSTM, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

Using TensorFlow backend.


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/krishna_primary/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/krishna_primary/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# %load "./preprocessing.py"

df = pd.read_csv("./final_train.csv")
test_df = pd.read_csv("./final_test.csv")
X = df['desc']
y = df['rating']

test_X = test_df['desc']

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

documents = []
test_documents = []

for i in range(len(test_df)):
	test_document = test_df['desc'][i].strip()

	test_document = test_document.lower()
	test_document = test_document.split()
	test_document = [word for word in test_document if not word in stop_words]	
	test_document = [lemmatizer.lemmatize(word) for word in test_document]
	test_document = [stemmer.stem(word) for word in test_document]
	test_document = ' '.join(test_document)
	test_documents.append(test_document)

for i in range(len(df)):
	document = df['desc'][i].strip()
	document = document.lower()
	document = document.split()
	document = [word for word in document if not word in stop_words]
	document = [lemmatizer.lemmatize(word) for word in document]
	document = [stemmer.stem(word) for word in document]
	document = ' '.join(document)
	documents.append(document)

output_train_df = pd.DataFrame({'desc': pd.Series(documents), 'rating': df['rating']})
output_test_df = pd.DataFrame({'Id': test_df['Id'], 'desc': pd.Series(test_documents)})

output_train_df.to_csv("preprocessed_final_train.csv", index=False)
output_test_df.to_csv("preprocessed_final_test.csv", index=False)





In [4]:
data = pd.read_csv('preprocessed_final_train.csv')
pd.set_option('display.max_colwidth',-1)
data.head()

desc  \
0  "humira gave life back. sicker ever first medicin consist made feel better. take conjunct pentasa seem magic formula me. don&#039;t like give shot grate feel. diet big part miracl me."                                                                                                                                                                                                                                                                                              
1  "i&#039;m terribl time hysingla. 40mg strong couldn&#039;t function. i&#039;v 30mg week. headaches, i&#039;m tire time, cramping. overal feel like crap. noth breakout pain tylenol. understand doctor prescrib abuse, but, taken what prescribed.. it&#039; bad enough i&#039;m actual forum complaining."                                                                                                                                                                           
2  "i four differ birth control pill past 2 years. got loryna 6 month ago, absolut love it. complet gotten rid acne, period 2 day now, little/ almost cramp pain. use terribl breakouts, noth would prevent help them. birth control clear skin 2nd week it, mayb zit (3 4) usual go away right away. birth control pill gave mood swings, mess periods, made acn worse. glad switch new doctor, put this. best!"                                                                        
3  "pain relief limit bare noticed. take 1800 mg day still need 6 percocet per day get enough relief function. i&#039;v tri stop gabapentin couldn&#039;t without withdrawals. forgotten take medicin long day also withdraw symptom includ sever stomach pain diarrhea. "                                                                                                                                                                                                               
4  "asid pocket cost ($130.00 me) would recommend drug. effect clean colon without drink copiou amount fluid. chose orang flavor remind powder tang orang juic - unpleas all. better chill possible. follow split dose regimen. took 90 minut first dose start working, process last 2 hours. experi slight, pass feel nausea first, subsid thing got moving. second dose, next morning, took amount time work, process last hour, physician report colon prep &quot;excellent&quot;."   

   rating  
0  10.0    
1  1.0     
2  10.0    
3  4.0     
4  9.0

In [5]:
print( "Number of reviews in data : ", len(data))

Number of reviews in data :  159523


In [6]:
num_words = 5000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(data['desc'].values)
X = tokenizer.texts_to_sequences(data['desc'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 150
X = pad_sequences(X, maxlen=max_length_of_text)

print(word_index)

Found 48130 unique tokens.
{'039': 1, 'i': 2, 't': 3, 'take': 4, 'day': 5, 'it': 6, 'get': 7, 'month': 8, 'start': 9, 'm': 10, 'work': 11, 'side': 12, 'pain': 13, 'feel': 14, 'year': 15, 'week': 16, 'time': 17, 'v': 18, 'effect': 19, 'first': 20, 'like': 21, '2': 22, 'use': 23, 'pill': 24, 'period': 25, '3': 26, 'go': 27, 'tri': 28, 'would': 29, 'help': 30, 'back': 31, 'doctor': 32, 'one': 33, 'took': 34, 'also': 35, 'weight': 36, 'got': 37, 'me': 38, 'life': 39, 'don': 40, 'bad': 41, 'still': 42, 'medic': 43, '5': 44, 'sinc': 45, 'much': 46, 'realli': 47, 'stop': 48, 'control': 49, 'never': 50, 'better': 51, 'went': 52, 'last': 53, '4': 54, 'two': 55, 'sleep': 56, 'quot': 57, 'sever': 58, 'felt': 59, 'even': 60, 'night': 61, 'well': 62, 'gain': 63, 'good': 64, 'everi': 65, 'didn': 66, 'great': 67, 'medicin': 68, '1': 69, 'drug': 70, 'make': 71, 'hour': 72, 'ago': 73, '6': 74, '10': 75, 'birth': 76, 'mg': 77, 'prescrib': 78, 'made': 79, 'effects': 80, 'thing': 81, 'anxieti': 82, 'dose'

In [7]:
X.shape

(159523, 150)

In [8]:
Y = pd.get_dummies(data['rating']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(119642, 150) (119642, 10)
(39881, 150) (39881, 10)


In [9]:
embed_dim = 250 
batch_size = 512

inputs = Input((max_length_of_text, ))
x = Embedding(num_words, embed_dim)(inputs)
x = LSTM(64)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs, x)
print(model.summary())




Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 250)          1250000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                80640     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,331,290
Trainable params: 1,331,290
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
model.compile(loss = 'categorical_crossentropy', optimizer='adamax', metrics = ['accuracy'])

In [16]:
model.fit(X_train, Y_train, epochs=45, batch_size=batch_size, validation_data=(X_test,Y_test))

Train on 119642 samples, validate on 39881 samples
Epoch 1/10
119642/119642 [==============================] - 218s 2ms/step - loss: 0.7032 - acc: 0.7719 - val_loss: 2.0104 - val_acc: 0.5139
Epoch 2/10
119642/119642 [==============================] - 222s 2ms/step - loss: 0.7008 - acc: 0.7729 - val_loss: 2.0458 - val_acc: 0.5143
Epoch 3/10
119642/119642 [==============================] - 222s 2ms/step - loss: 0.6652 - acc: 0.7873 - val_loss: 2.0592 - val_acc: 0.5187
Epoch 4/10
119642/119642 [==============================] - 236s 2ms/step - loss: 0.6546 - acc: 0.7903 - val_loss: 2.0975 - val_acc: 0.5206
Epoch 5/10
119642/119642 [==============================] - 235s 2ms/step - loss: 0.6401 - acc: 0.7955 - val_loss: 2.1224 - val_acc: 0.5205
Epoch 6/10
119642/119642 [==============================] - 230s 2ms/step - loss: 0.6338 - acc: 0.7979 - val_loss: 2.1391 - val_acc: 0.5239
Epoch 7/10
119642/119642 [==============================] - 239s 2ms/step - loss: 0.6256 - acc: 0.8009 - val_

In [17]:
score, acc = model.evaluate(X_test, Y_test, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

39881/39881 [==============================] - 19s 473us/step
Score: 2.23
Validation Accuracy: 0.53


In [18]:
test_df = pd.read_csv("preprocessed_final_test.csv")
test_df.head()

Id  \
0  0    
1  1    
2  2    
3  3    
4  4    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             desc  
0  "klonopin great anxieti agit associ bi-polar symptom even lowest 0.5mg dose. "                                                                                                                                                                                                                                                                                                                                                                                                                  
1  "i 20 year old. i&#039;v moder acn sinc 13. i&#039;v never complet clear skin sinc age. need get birth control bit nervou side effects. chose yaz claim said would help clear acne. start first pack breast becam sore slightli nauseous. noth seem happen skin. second pack start break out, wasn&#039;t sure birth control me. nausea went away. fourth pack breakout worst. frustrat want quit birth control. end fourth pack notic acn clearing. fifth month coupl small pimpl hope high!"  
2  "to earli tell."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
3  "i hospit (9 time 2 years). best medicin ever used. hate feel pain medicin give you. one made sleepi pain free. bad wouldn&#039;t let take home. fentanyl patch instead."                                                                                                                                                                                                                                                                                                                       
4  "i&#039;v use gianvi month haven&#039;t experienc mood swings!!! don&#039;t feel depressed, suicid anything. experienc bloat tender breasts. recent broke face bit i&#039;m hope bodi soon adjust chose help acne. i&#039;ll continu use hope best. far don&#039;t think i&#039;v gain weight bummer bc i&#039;m underweight realli look forward it. we&#039;ll see!"

In [19]:
sequences_test = tokenizer.texts_to_sequences(test_df["desc"])

x_pred = pad_sequences(sequences_test, maxlen=150)
y_pred = model.predict(x_pred)

# print(y_pred)

y_pred = [np.argmax(i)+1 for i in y_pred]

# print(y_pred)

output_df = pd.DataFrame({'Id': test_df['Id'], 'Rating': pd.Series(y_pred)})
output_df.to_csv("lstm_submission_1.csv", index=False)

In [20]:
model.save("model.h5")